<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Understanding Ocean Deoxygenation</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [1]:
# ---------- Librairies ----------
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr

# ---------- Jupyter ----------
%matplotlib inline
plt.rcParams.update({"font.size": 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
%run scripts/run_training.py -b async -c scripts/configs/debug.yml

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [1]:
import torch
from poseidon.data.const import TOY_DATASET_REGION
from poseidon.diffusion.backbone import PoseidonBackbone
from poseidon.diffusion.denoiser import PoseidonDenoiser

# Dummy configuration
config_unet = {
    "kernel_size": 5,
    "mod_features": 1,
    "hid_channels": [256, 256, 384],
    "hid_blocks": [3, 3, 3],
    "dropout": 0.1,
    "attention_heads": {"4" : 8}
}

config_siren = {
    "features": 2,
    "n_layers": 1,
}

config_problem = {
    "toy_problem": True
}


# Testing number of parameters
B, C, T, H, W = 1, 4, 5, 128, 256


# Creating a dummy denoiser
backbone = PoseidonBackbone(
    dimensions=(B, C, T, H, W),
    config_unet=config_unet,
    config_siren=config_siren,
    config_region=TOY_DATASET_REGION,
)

denoiser = PoseidonDenoiser(backbone=backbone).to("cuda")
print("Number of parameters [1e6]:", sum(p.numel() for p in denoiser.parameters())/1e6)

# ===== Testing Forward Pass =====
x = torch.randn(B, (C * T * H * W)).to("cuda")
sigma = torch.randn(B, 1).to("cuda")
y = denoiser(x, sigma)

print("Input shape:", x.shape)
print("Output shape:", y.shape)


Number of parameters [1e6]: 475.314556
Input shape: torch.Size([1, 655360])
Output shape: torch.Size([1, 655360])


In [ ]:
Number of parameters [1e6]: 0.681432
Number of parameters [1e6]: 0.553432
Number of parameters [1e6]: 0.524632
Number of parameters [1e6]: 0.486232

In [ ]:
512 + 128

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Example tensor shape
B, C, T, H, W = 2, 16, 10, 32, 32  # (Batch, Channels, Time, Height, Width)
x = torch.randn(B, C, T, H, W)  # Input tensor

# Define convolution (without built-in padding)
K = 3  # Kernel size along T
conv = nn.Conv3d(in_channels=C, out_channels=C, kernel_size=(K, 1, 1), stride=(1, 1, 1), padding=(0, 0, 0))

# Apply circular padding only on the T dimension
pad_size = K // 2  # How much to pad on each side of T
x_padded = F.pad(x, (0, 0, 0, 0, pad_size, pad_size), mode="circular")  # Pads only T

print(x[0,0,:,0,0])
print(x_padded[0,0,:,0,0])
print(x_padded.shape)

# Apply convolution
output = conv(x_padded)

# Ensure the output has the same T dimension
print(output.shape)  # Expected: (B, C, T, H, W)
